# Data Visualization Demo
### A quick demo of the current binary tree model
Analysis set up:

In [1]:
import sys
sys.path.append('/Users/tyler/Projects/latin-prose-rhythm')

from prose_rhythm.analyze import Analyze
from prose_rhythm.preprocessor import Preprocessor
from prose_rhythm.binary_tree import Tree
import networkx as nx

Read-in Caesar's Civil War:

In [2]:
caesar_civil_war_path = "/Users/tyler/Projects/macronized_texts/Caesar_CivilWar_1-22-18.txt"
with open(caesar_civil_war_path, 'r') as file:
    text = file.read()
text[:100]

' COMMENTARIORVM BELLĪ CĪVĪLIS LĪBER PRIMVS Litterīs ā Fabiō C. Caesaris cōnsulibus redditīs aegrē ab'

Process the text into structured tokens:

In [3]:
preprocessor = Preprocessor(text)
tokens = preprocessor.tokenize()
tokens["text"][0]

{'contains_abbrev': True,
 'contains_numeral': False,
 'plain_text_sentence': 'commentariorvm bellī cīvīlis līber primvs litterīs ā fabiō caesaris cōnsulibus redditīs aegrē ab hīs impetrātum est summa tribūnōrum plēbis contentiōne ut in senātū recitārentur',
 'structured_sentence': [{'index': 0,
   'syllables': [{'accented': False,
     'elide': (False, None),
     'index': 0,
     'long_by_nature': False,
     'long_by_position': (True, None),
     'syllable': 'com'},
    {'accented': False,
     'elide': (False, None),
     'index': 1,
     'long_by_nature': False,
     'long_by_position': (True, None),
     'syllable': 'men'},
    {'accented': True,
     'elide': (False, None),
     'index': 2,
     'long_by_nature': False,
     'long_by_position': (False, None),
     'syllable': 'ta'},
    {'accented': False,
     'elide': (False, None),
     'index': 3,
     'long_by_nature': False,
     'long_by_position': (False, None),
     'syllable': 'ri'},
    {'accented': False,
     'elide

Create a list of rhythms from tokens:

In [4]:
analyze = Analyze(include_short_clausula=False)
rhythm_list = Analyze().get_rhythms(tokens)
print(len(rhythm_list)) # Total number of rhythms
rhythm_list[:5] # First 5 rhythms

1064


['u-u---ux', '-u---u-x', 'u-u----x', '----u--x', '---u---x']

Create binary tree from rhythm list:

In [5]:
tree = Tree()
for rhythm in rhythm_list:
    tree.add(rhythm)

Calculate probability of rhythm in rhythm tree:

In [6]:
tree.calculate_probability()

Create JSON object from tree for visualization:

In [7]:
json_tree = tree.to_json()
json_tree

'{"name": "x 0", "children": [{"name": "u 0.30733", "children": [{"name": "u 0.29969", "children": [{"name": "u 0.33673", "children": [{"name": "u 0.21212", "children": [{"name": "u 0.42857", "children": [{"name": "- 1.0", "children": [{"name": "u 0.66667", "children": []}, {"name": "- 0.33333", "children": []}]}]}, {"name": "- 0.57143", "children": [{"name": "u 0.75", "children": [{"name": "u 0.33333", "children": []}, {"name": "- 0.66667", "children": []}]}, {"name": "- 0.25", "children": [{"name": "u 1.0", "children": []}]}]}]}, {"name": "- 0.78788", "children": [{"name": "u 0.53846", "children": [{"name": "u 0.14286", "children": [{"name": "- 1.0", "children": []}]}, {"name": "- 0.85714", "children": [{"name": "u 0.5", "children": []}, {"name": "- 0.5", "children": []}]}]}, {"name": "- 0.46154", "children": [{"name": "u 0.33333", "children": [{"name": "u 0.25", "children": []}, {"name": "- 0.75", "children": []}]}, {"name": "- 0.66667", "children": [{"name": "u 0.375", "children": 

In [8]:
%load_ext py_d3

In [9]:
%ls

Tree_Demo.ipynb     cicero_test.ipynb   tacitus_test.ipynb
caesar_test.ipynb   pliny_test.ipynb


In [10]:
%%d3

<style> /* set the CSS */
body {
  background-color: #ccc
}

.node circle {
  fill: #fff;
  stroke: steelblue;
  stroke-width: 3px;
}

.node text { font: 12px sans-serif; }

.node--internal text {
  text-shadow: 0 1px 0 #fff, 0 -1px 0 #fff, 1px 0 0 #fff, -1px 0 0 #fff;
}

.link {
  fill: none;
  stroke: #000;
  stroke-width: 2px;
}

</style>

<g></g>
<script>

// set the dimensions and margins of the diagram
var margin = {top: 20, right: 90, bottom: 30, left: 90},
  width = 1000 - margin.left - margin.right,
  height = 4500 - margin.top - margin.bottom;

// declares a tree layout and assigns the size
var treemap = d3.tree()
  .size([height, width]);

// load the external data
d3.json("../data/caesar_civil_war.json", function(error, treeData) {
  if (error) throw error;

  //  assigns the data to a hierarchy using parent-child relationships
  var nodes = d3.hierarchy(treeData, function(d) {
    return d.children;
    });

  // maps the node data to the tree layout
  nodes = treemap(nodes);

  // append the svg object to the body of the page
  // appends a 'group' element to 'svg'
  // moves the 'group' element to the top left margin
  var svg = d3.select("body").append("svg")
      .attr("width", width + margin.left + margin.right)
      .attr("height", height + margin.top + margin.bottom),
    g = svg.append("g")
      .attr("transform",
        "translate(" + margin.left + "," + margin.top + ")");

  // adds the links between the nodes
  var link = g.selectAll(".link")
    .data( nodes.descendants().slice(1))
    .enter().append("path")
    .attr("class", "link")
    .attr("d", function(d) {
       return "M" + d.y + "," + d.x
       + "C" + (d.y + d.parent.y) / 2 + "," + d.x
       + " " + (d.y + d.parent.y) / 2 + "," + d.parent.x
       + " " + d.parent.y + "," + d.parent.x;
       });

  // adds each node as a group
  var node = g.selectAll(".node")
    .data(nodes.descendants())
    .enter().append("g")
    .attr("class", function(d) { 
      return "node" + 
      (d.children ? " node--internal" : " node--leaf"); })
    .attr("transform", function(d) { 
      return "translate(" + d.y + "," + d.x + ")"; });

  // adds the circle to the node
  node.append("circle")
    .attr("r", 10);

  // adds the text to the node
  node.append("text")
    .attr("dy", ".35em")
    .attr("x", function(d) { return d.children ? -13 : 13; })
    .style("text-anchor", function(d) { 
    return d.children ? "end" : "start"; })
    .text(function(d) { return d.data.name; });

});
</script>
